## tp2

### Extraction de mots-clefs avec Yake

In [ ]:
import yake
import os

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
#création d'un nouveau repo consacré aux fichiers séléctionnés de l'année 1893
!mkdir ../data/txt/txt_1893
!cp ../data/txt/KB_JB838_1893-*.txt ../data/txt/txt_1893
!wc -l ../data/txt/txt_1893/*

In [ ]:
# Lister les Fichiers
data_path = "../data/txt/txt_1893"
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]
len(files)

In [ ]:
# Choisir un fichier
this_file = files[0]
this_file

In [ ]:
# Récupérer le texte du fichier
text = open(os.path.join(data_path, this_file), 'r', encoding='utf-8').read()
text[:500]

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

In [ ]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

In [ ]:
## Réaliser la même opérations sur tous les documents

In [ ]:
for f in sorted(files)[:100]:
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

### Nuage de mots

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "sach", "heures", "toutes", "hier", "très"
       , "etc", "leurs", "grandes", "lés", "ille", "quo", "peu", "bon", "vers", "grand", "puis", "mois", "personne",
       "devant", "beau", "mén", "elles", "toujours", "déjà", "avenue", "quatre", "fort", "jours", "aujourd",
       "car", "hier", "toute", "grands", "app", "matin", "bne", "bas", "adresser", "haut", "dés",
       "place", "rue", "Bruxelles", "DCM", "ecr", "jour","maison", "ans", "chez", "réf", "prendre",
       "mod", "pers", "suite", "Mme", "voir", "cause", "louer", "porte", "belle", "fille",
       "adr", "quart", "maisons", "rué", "fit", "seule", "agence", "plusieurs", "bonnes", "ceux", "quelques",
       "faits", "jeune", "cent", "vient", "point", "portées", "avant", "serv", "com", "jard",
       "enfant", "désire", "cuis", "Bruxelles", "bruxelles", "grande", "petit", "bons", "dcm", "soir",
       "près", "bonne", "demande", "prix", "fois", "dés", "vendre", "jamais", "chaussée", "bonne", "franc", "lieu", "rien", "quartier",
       "pris", "fois", "tranq", "pos", "vente", "bonne", "neuve", "cuisine", "servante", "mat", "écrire",
       "juillet", "notaire", "dès", "quand", "temps", "pet", "celui", "donné", "partie", "homme", "petite", "cours",
       "mme","part", "coup", "demi", "pens", "neuf", "ruo", "cinq", "ici", "cond", "ferme", "coucher", "demandé",
       "ferm", "comm", "notaires", "août", "prop", "alors", "mardi", "beaucoup", "enfants", "ventes", "lundi", "jeudi",
       "donner", "nuit", "chamb", "chaque", "dimanche", "dos", "uno", "garn", "lés", "fam", "jolie", "demain", "année",
       "petite", "fam", "uno", "celui", "contenant", "premier", "assez", "nouvelles", "beaucoup", "cond", "con",
       "PET", "vendredi", "quand", "nouveau", "gros", "dix", "mieux", "journ", "chaque", "cert", "dém", "dos",
       "vend", "chamb", "alors", "seulement", "mars", "delà", "trop", "courant", "demain", "brux",
       "dernier", "hon", "bel", "seul", "frais", "petits", "dernière", "mis", "diverses", "référ",
       "jne", "tant", "franco", "francs","quelque", "nouvelle", "vieux", "placé", "garni",
       "mal", "sachant", "six", "nommé", "propre", "avril", "coud", "juin", "octobre", "enf",
       "mis", "divers", "sait", "servi", "flam", "septembre", "frais", "hom", "bel", "hon",
       "ouvrir", "enfin", "février", "hui", "dessus", "lès", "janvier", "scs", "partout", "façon",
       "laquelle", "ste", "samedi", "mans", "années", "ment", "mai", "voici", "céder", "surtout",
       "mercredi", "parmi", "franç", "font", "ancien", "cond", "mlle", "jeunes", "beaux", "semaine",
       "garnie", "reçu", "trouve", "derrière", "ruc", "rest", "conn", "lit", "haute", "parce", "fin",
       "aucune", "aussitôt", "ouvr", "aucune"]
sw = set(sw)

In [ ]:
# Choisir une année
year = 1893

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

In [ ]:
#fonction de nettoyage
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

In [ ]:
#afficher les termes les + fréquents
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
#Créer; afficher et stocker le wordcloud
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

### Reconnaissance d'entités nommées avec SpaCy

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
# Charger le texte
n=100000
text = open("../data/tmp/1893.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:

# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

In [ ]:
# Compter les entités
location = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        location[ent.text] += 1
        sorted_location = sorted(location.items(), key=lambda kv: kv[1], reverse=True)

for location, freq in sorted_location[:50]:
    print(f"{location} apparait {freq} fois dans le corpus")

In [ ]:
# Compter les entités
org = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "ORG" and len(ent.text) > 3:
        org[ent.text] += 1
        sorted_org = sorted(org.items(), key=lambda kv: kv[1], reverse=True)

for org, freq in sorted_org[:50]:
    print(f"{org} apparait {freq} fois dans le corpus")

### Sentiment Analysis

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

In [ ]:
!pip install tensorflow
!pip install sentencepiece
!pip install transformers

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
sentiment_analyser("Les funérailles du maire Alexiéw, aux frais de la ville, ont été graiidioses")

In [ ]:
sentiment_analyser("Les funérailles du maire Alexiéw, aux frais de la ville, ont été grandioses")

In [ ]:
sentiment_analyser("Paradoxes et vérités': . . Trop d’esprit humilie ceux qui n’en ont pas.")

In [ ]:
sentiment_analyser("n homme a ouvert la porte, un mulâtre de taille assez corpulente, aux cheveux déjà couleur de neige, aux yeux jaunes, aux dents blanches qui font paraître sa figura basanéo encore plus noire. ")

In [ ]:
sentiment_analyser("Èt ainsi pendant des années on « étudiera » lé redressement de la Montagne de la Cour")

In [ ]:
sentiment_analyser(" L’histoire de son «sauvetage», qui ressemblait fort à fine capture, est encore dans tôutès les mémoires et l’on n’a pas oublié l’accident tragique »— une chute parla fenêtre — qui, à Bagamoyô, au moment où on le rendait de force aux délices de la civilisation, faillit coûter la vie à celui que. les fatigues, les privations, les maladies, la rébellion même avaient épargné : comme si l’Afrique, à laquelle on l’arrachait, voulait le faire souvenir de çe nom (le « Fidèle « qu’ii avait pris en venant vivre sur son sol. ")

In [ ]:
sentiment_analyser("Des personnes, dont les préoccupations patriotiques exagèrent peut-être les craintes, ariirment que l’outillage de celto. manufacture est défectueux.")

In [ ]:
sentiment_analyser("Les', grévistes.' se sont réfugiés dans les cours et dansjes jardins ét de là ont fait pleuvoir sur les gendarmes ùnegrèle de pierres.")

In [ ]:
sentiment_analyser("Les mesures prises par le gouvernement pour arrêter le mouvement d’extension de la grève vont être renforcées par l’envoi, demain lundi, de nouveaux régiments dans lé Hainaut et à Garni.")

In [ ]:
sentiment_analyser("Il parait que la garnison de Gand sera renforcée par de la cavalerie, un régiment entier d’infanterie et même du génie.")

In [ ]:
sentiment_analyser("On continue à amener à la prison de Mous les grévistes arrêtés ,pour atteinte à la liberté du travail et pour coups aux gendarmes.")